In [90]:
from unet_model import *
from glob import glob
import random
import matplotlib.pyplot as plt
from pathlib import Path
from torchvision import transforms
from torchvision.transforms import InterpolationMode
from datetime import datetime
from collections import namedtuple
import numpy as np
from PIL import Image
from ignite.metrics import ConfusionMatrix, mIoU
from ignite.engine import Engine
from torchmetrics import JaccardIndex

In [62]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#device = "cpu"
print(device)

train_path = glob('/cluster/projects/vc/data/ad/open/Cityscapes/leftImg8bit_trainvaltest/leftImg8bit/train/*/*leftImg8bit.png')
val_path = glob('/cluster/projects/vc/data/ad/open/Cityscapes/leftImg8bit_trainvaltest/leftImg8bit/val/*/*leftImg8bit.png')
len(val_path)

cuda


500

In [188]:
GroupedLabel = namedtuple("GroupedLabel", [
    "id", # group id
    "name", # group name
    "ids", # list of ids
    "color", # color of the group
])
# grouped_labels = [
#     GroupedLabel(0, "motor vehicles" , [26, 27, 28, 28, 29, 30, 31, 32],           (  0,   0, 142)),
#     GroupedLabel(1, "pedestrians"    , [24, 25, 33],                               (220,  20,  60)),
#     GroupedLabel(2, "road"           , [6, 7, 8, 9, 10],                           (128,  64, 128)),
#     GroupedLabel(3, "traffic objects", [17, 18, 19, 20],                           (250, 170,  30)),
#     GroupedLabel(4, "background"     , [4, 5, 11, 12, 13, 14, 15, 16, 21, 22, 23], ( 70,  70,  70)),
#     GroupedLabel(5, "void"           , [0, 2, 3],                                  (  0,   0,   0)),
#     GroupedLabel(6, "ego vehicle"    , [1],                                        (  0,   0,   0)),
# ]
grouped_labels = [
    GroupedLabel(id=0, name='unlabeled', ids=[0, 1, 2, 3, 4], color=(0, 0, 0)),
    GroupedLabel(id=1, name='dynamic', ids=[5], color=(111, 74, 0)),
    GroupedLabel(id=2, name='ground', ids=[6], color=(81, 0, 81)),
    GroupedLabel(id=3, name='road', ids=[7], color=(128, 64, 128)),
    GroupedLabel(id=4, name='sidewalk', ids=[8], color=(244, 35, 232)),
    GroupedLabel(id=5, name='parking', ids=[9], color=(250, 170, 160)),
    GroupedLabel(id=6, name='rail track', ids=[10], color=(230, 150, 140)),
    GroupedLabel(id=7, name='building', ids=[11], color=(70, 70, 70)),
    GroupedLabel(id=8, name='wall', ids=[12], color=(102, 102, 156)),
    GroupedLabel(id=9, name='fence', ids=[13], color=(190, 153, 153)),
    GroupedLabel(id=10, name='guard rail', ids=[14], color=(180, 165, 180)),
    GroupedLabel(id=11, name='bridge', ids=[15], color=(150, 100, 100)),
    GroupedLabel(id=12, name='tunnel', ids=[16], color=(150, 120, 90)),
    GroupedLabel(id=13, name='pole', ids=[17, 18], color=(153, 153, 153)),
    GroupedLabel(id=14, name='traffic light', ids=[19], color=(250, 170, 30)),
    GroupedLabel(id=15, name='traffic sign', ids=[20], color=(220, 220, 0)),
    GroupedLabel(id=16, name='vegetation', ids=[21], color=(107, 142, 35)),
    GroupedLabel(id=17, name='terrain', ids=[22], color=(152, 251, 152)),
    GroupedLabel(id=18, name='sky', ids=[23], color=(70, 130, 180)),
    GroupedLabel(id=19, name='person', ids=[24], color=(220, 20, 60)),
    GroupedLabel(id=20, name='rider', ids=[25], color=(255, 0, 0)),
    GroupedLabel(id=21, name='car', ids=[26, -1], color=(0, 0, 142)),
    GroupedLabel(id=22, name='truck', ids=[27], color=(0, 0, 70)),
    GroupedLabel(id=23, name='bus', ids=[28], color=(0, 60, 100)),
    GroupedLabel(id=24, name='caravan', ids=[29], color=(0, 0, 90)),
    GroupedLabel(id=25, name='trailer', ids=[30], color=(0, 0, 110)),
    GroupedLabel(id=26, name='train', ids=[31], color=(0, 80, 100)),
    GroupedLabel(id=27, name='motorcycle', ids=[32], color=(0, 0, 230)),
    GroupedLabel(id=28, name='bicycle', ids=[33], color=(119, 11, 32))
 ]

def classes_to_rgb(output: torch.Tensor) -> torch.Tensor:
    # Input: (num_classes, H, W)
    # Output: (3, H, W)
    rgb = torch.zeros((3, *output.size()[-2:]))
    output_max = torch.argmax(output.squeeze(), dim=0)
    for label in grouped_labels:
        for c in range(3):
            rgb[c][output_max == label.id] = label.color[c] / 255
    return rgb

def gt_to_classes(gt: np.ndarray) -> torch.Tensor:
    # Input: (H, W), values: 0-num_classes
    # Output: (num_classes, H, W)
    output = torch.zeros((len(grouped_labels), *gt.shape))
    for group in grouped_labels:
        output[group.id] = torch.Tensor(np.isin(gt, group.ids))
    return output

def gt_to_masks(gt: np.ndarray) -> torch.Tensor:
    # Input: (H, W), values: 0-num_classes
    # Output: (num_classes, H, W)
    pass

def infer(unet_model, img_path, out_path):
    label_path = img_path.replace("leftImg8bit", "gtFine").replace(".png", "_labelIds.png")
    img = np.array(Image.open(img_path))
    label = np.array(Image.open(label_path))
    h, w = img.shape[:2]

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((h//2, w//2), antialias=True),
        #transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),
    ])
    transform1 = transforms.Compose([
        transforms.Resize((h//2, w//2), antialias=True),
    ])

    img_in = transform(img).to(device)[None]
    label_in = transform1(gt_to_classes(label)).to(device)[None]
    output = unet_model(img_in)
    output = classes_to_rgb(output)
    label_in = classes_to_rgb(label_in)
    with torch.no_grad():
        output = output.cpu().detach().numpy().transpose(1, 2, 0).clip(0, 1)
    img_in = img_in.cpu().detach()[0].numpy().transpose(1, 2, 0)
    label_in = label_in.cpu().detach().numpy().transpose(1, 2, 0)
    
    plt.imsave(out_path, output)
    return img_in, output, label_in
    # fig, ax = plt.subplots(1, 3, figsize=(20, 5))
    # ax[0].imshow(img_in)
    # ax[1].imshow(output)
    # ax[2].imshow(label)
    # plt.show()

def infer_metric(unet_model, img_path):
    label_path = img_path.replace("leftImg8bit", "gtFine").replace(".png", "_labelIds.png")
    img = np.array(Image.open(img_path))
    label = np.array(Image.open(label_path))
    h, w = img.shape[:2]

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((h//2, w//2), antialias=True),
        #transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),
    ])
    transform1 = transforms.Compose([
        #transforms.ToTensor(),
        transforms.Resize((h//2, w//2), antialias=True, interpolation=InterpolationMode.NEAREST),
    ])

    img_in = transform(img).to(device)[None]
    label_in = transform1(gt_to_classes(label)).long().to(device)[None]
    #label_in = transform1(label).long().to(device)#[None]

    with torch.no_grad():
        output = unet_model(img_in)
        argmax = torch.argmax(output.squeeze(), dim=0)
        ious = []
        intersections = 0
        unions = 0
        for i in range(len(grouped_labels)):
            output[0][i] = argmax == i
            pred = argmax == i
            y = label_in[0][i]
            intersection = torch.bitwise_and(pred, y).sum().item()
            union = torch.bitwise_or(pred, y).sum().item()
            intersections += intersection
            unions += union
            if union == 0:
                union = 1
            if intersection:
                iou = intersection/union
                #print(iou)
                ious.append(iou)
        iou = sum(ious) / len(ious)
        accuracy = intersections / (label_in.size()[-1] * label_in.size()[-2])
        # print(label_in.size()[-1] * label_in.size()[-2])
        # print(accuracy)
        # print()
        print(intersections / unions)
        # print()
        print(iou)
    
    # default_evaluator = Engine(lambda e, b: b)
    # cm = ConfusionMatrix(num_classes=len(grouped_labels))
    # metric = mIoU(cm, ignore_index=0)
    # metric.attach(default_evaluator, 'miou')
    # state = default_evaluator.run([[output, label_in]])
    # miou = state.metrics['miou']
    
    #jaccard = JaccardIndex(task="multilabel", num_labels=len(grouped_labels)).to(device)
    jaccard = JaccardIndex(task="multiclass", num_classes=len(grouped_labels)).to(device)
    iou = jaccard(output, label_in)
    miou = iou.item()

    return miou, accuracy

def infer_output(unet_model, img_path, out_path):
    img = np.array(Image.open(img_path).convert("RGB"))

    transform = transforms.Compose([
        transforms.ToTensor(),
        #transforms.Resize((1024, 512)), antialias=True),
    ])

    img_in = transform(img).to(device)[None]
    with torch.no_grad():
        output = unet_model(img_in)
    output = classes_to_rgb(output)
    output = output.cpu().detach().numpy().transpose(1, 2, 0).clip(0, 1)
    img_in = img_in.cpu().detach()[0].numpy().transpose(1, 2, 0)
    
    plt.imsave(out_path, output)

In [192]:
models = Path("weights").glob("unetsegment_231127_031540_checkpoint_*.pt")
models = sorted(models, key=lambda x: int(x.name.split("_")[-1].split(".")[0]), reverse=True)

#img = random.choice(val_path)
for model in models:
    #print(f"Saving {model} output")
    unet_model = UNet(3, len(grouped_labels)).float().to(device)
    checkpoint = torch.load(model)
    unet_model.load_state_dict(checkpoint['model_state_dict'])
    #_, _, label = infer(unet_model, img, f"output/{model.stem}.png")
    mious = []
    accs = []
    random.shuffle(val_path)
    for img in val_path:
        miou, accuracy = infer_metric(unet_model, img)
        mious.append(miou)
        accs.append(accuracy)
        print(np.array(mious).mean(), np.array(mious).std())
        #print(np.array(accs).mean(), np.array(accs).std())
        break
    break
#plt.imsave(f"output/{model.stem}_gt.png", label)

0.7782571460066309
0.5786729949031086
0.8846948146820068 0.0


In [34]:
model = "weights/unetsegment_231127_031540_checkpoint_245.pt"
img = Path("misc").joinpath("Trondheim5.png")
unet_model = UNet(3, len(grouped_labels)).float().to(device)
checkpoint = torch.load(model)
unet_model.load_state_dict(checkpoint['model_state_dict'])
infer_output(unet_model, img, f"misc/{img.stem}_out.png")

In [29]:
! ffmpeg -y -f image2 -i "output/unetsegment_231127_031540_checkpoint_%d.png" -c:v libx264 -vf "setpts=(3.0-0.01*N)*PTS,hqx=2" -r 30 -pix_fmt yuv420p -crf 13 -x264-params psy-rd=0 output/video_{datetime.now().strftime(r"%y%m%d_%H%M%S")}.mp4

ffmpeg version 6.1-static https://johnvansickle.com/ffmpeg/  Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 8 (Debian 8.3.0-6)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-debug --disable-ffplay --disable-indev=sndio --disable-outdev=sndio --cc=gcc --enable-fontconfig --enable-frei0r --enable-gnutls --enable-gmp --enable-libgme --enable-gray --enable-libaom --enable-libfribidi --enable-libass --enable-libvmaf --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librubberband --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libvorbis --enable-libopus --enable-libtheora --enable-libvidstab --enable-libvo-amrwbenc --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libdav1d --enable-libxvid --enable-libzvbi --enable-libzimg
  libavutil      58. 29.100 / 58. 29.100
  libavcodec     60. 31.102 / 60. 31.102
  libavformat    6